In [16]:
import tensorflow as tf

#y = Wx + b
x_train = [1.0,2.0,3.0,4.0]
y_train = [-1.0,-2.0,-3.0,-4.0]

W = tf.Variable(initial_value=[1.0], dtype=tf.float32, name='W')
b = tf.Variable(initial_value=[1.0], dtype=tf.float32, name='b')
x = tf.placeholder(dtype=tf.float32, name='x')

y_input = tf.placeholder(dtype=tf.float32, name='y_input')
#y_output = W * x + b
y_output = tf.add(x=tf.multiply(x=W, y=x,name='multiply'), y=b, name='y_output')



In [17]:
loss = tf.reduce_sum(input_tensor=tf.square(x=y_output - y_input), name='loss')
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01, name='optimizer')
train_step = optimizer.minimize(loss=loss, name='train_step')

saver = tf.train.Saver()

session = tf.Session()
session.run(tf.global_variables_initializer())


tf.train.write_graph(graph_or_graph_def=session.graph_def, 
                     logdir='.', 
                     name='linear_regression.pbtxt', 
                     as_text=False)
# Print loss before training
print(session.run(fetches=loss, feed_dict={x:x_train, y_input:y_train}))

for _ in range(1000):
    session.run(fetches = train_step, feed_dict={x:x_train, y_input:y_train})

saver.save(sess = session,
          save_path='linear_regression.ckpt')
# Print loss, w and b after training
print(session.run(fetches=[loss, W, b], feed_dict={x:x_train, y_input:y_train}))

# Test the model
print("testing model")
print(session.run(fetches=y_output, feed_dict={x:[5.0,10.0]}))

164.0
[3.1441516e-12, array([-1.0000007], dtype=float32), array([2.0641437e-06], dtype=float32)]
testing model
[ -5.000002 -10.000006]


In [18]:
# Freeze Graph

In [19]:
from tensorflow.python.tools import freeze_graph, optimize_for_inference_lib

In [20]:
freeze_graph.freeze_graph(input_graph='linear_regression.pbtxt',
                         input_saver='',
                         input_binary=True,
                         input_checkpoint='linear_regression.ckpt',
                         output_node_names='y_output',
                         restore_op_name='save/restore_all',
                         filename_tensor_name='save/Const:0',
                         output_graph='frozen_linear_regression.pb',
                         clear_devices=True,
                         initializer_nodes='',
                         variable_names_blacklist='')

input_graph_def = tf.GraphDef()
with tf.gfile.Open('frozen_linear_regression.pb', 'rb') as f:
    data = f.read()
    input_graph_def.ParseFromString(data)
    
output_graph_def = optimize_for_inference_lib.optimize_for_inference(input_graph_def=input_graph_def,
                                                                        input_node_names=['x'],
                                                                        output_node_names=['y_output'],
                                                                        placeholder_type_enum=tf.float32.as_datatype_enum)
    
f = tf.gfile.FastGFile(name='optimized_frozen_linear_regression.pb',
                      mode='w')

f.write(file_content=output_graph_def.SerializeToString())

INFO:tensorflow:Restoring parameters from linear_regression.ckpt
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.
